# Lecture 4 - Julia - Types, Methods and Multiple Dispatch

## Julia, The Story So Far

- In lecture 1, we introduced the basics of the Julia programming language, including arrays, functions, and basic flow control.
- In lecture 2, we introduced Pkg.jl, the Julia package manager, as well as the library DifferentialEquations.jl.
- Today, we'll "remove the training wheels" and discuss Julia's type system in a lot more detail than before.

## Contents
1. [Types](#types)
2. [Composite Types](#composite-types)
3. [Parametric Types](#parametric-types)
4. [Methods and Multiple Dispatch](#methods)
5. [Parametric Methods](#parametric-methods)
6. [Conclusion](#conclusion)
7. [Further Reading](#further-reading)
8. [Next Week](#next-week)
9. [Quick Survey](#survey)

In [7]:
# Create a temporary environment and add a couple of useful packages for this lecture
using Pkg
Pkg.activate(; temp = true)
Pkg.add(["BenchmarkTools", "AbstractTrees"])

  Activating new project at `/tmp/jl_KWvCQ4`
   Resolving package versions...
    Updating `/tmp/jl_KWvCQ4/Project.toml`
  [1520ce14] + AbstractTrees v0.4.3
  [6e4b80f9] + BenchmarkTools v1.3.2
    Updating `/tmp/jl_KWvCQ4/Manifest.toml`
  [1520ce14] + AbstractTrees v0.4.3
  [6e4b80f9] + BenchmarkTools v1.3.2
  [682c06a0] + JSON v0.21.3
  [69de0a69] + Parsers v2.5.0
  [66db9d55] + SnoopPrecompile v1.0.1
  [56f22d72] + Artifacts
  [ade2ca70] + Dates
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [a63ad114] + Mmap
  [de0858da] + Printf
  [9abbd945] + Profile
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
  [cf7118a7] + UUIDs
  [4ec0a83e] + Unicode
  [e66e0078] + CompilerSupportLibraries_jll v0.5.2+0
  [4536629a] + OpenBLAS_jll v0.3.20+0
  [8e850b90] + libblastrampoline_jll v5.1.1+0


In [8]:
using BenchmarkTools, AbstractTrees

# Useful code snippet for printing type trees
AbstractTrees.children(x::Type) = subtypes(x)

## 1. Types <a class="anchor" id="types"></a>

- Every value in Julia is a first-class object, and **all objects have a type**.


- In the words of the Julia documentation,


> There is no division between object and non-object values: all values in Julia are true objects having a type that belongs to a single, fully connected type graph, all nodes of which are equally first-class as types.


- What do we mean when we're talking about a fully connected type graph?

In [9]:
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


- Why do we need a [type system](https://en.wikipedia.org/wiki/Type_system) at all? Types are essential in specifying the **structure and behaviour** of our program, both at a high level (the code we write in Julia) and at a low level (the underlying actions taken by our computer when we run our code).

In [10]:
typeof(1.0)

Float64

In [11]:
typeof(1)

Int64

In [12]:
bitstring(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

In [13]:
bitstring(1)

"0000000000000000000000000000000000000000000000000000000000000001"

In [14]:
@code_native binary = true 1 + 1

	.text
	.file	"+"
	.globl	"julia_+_3126"                  # -- Begin function julia_+_3126
	.p2align	4, 0x90
	.type	"julia_+_3126",@function
"julia_+_3126":                         # @"julia_+_3126"
; ┌ @ int.jl:87 within `+`
	.cfi_startproc
# %bb.0:                                # %top
	leaq	(%rdi,%rsi), %rax               # encoding: [0x48,0x8d,0x04,0x37]
	retq                                    # encoding: [0xc3]
.Lfunc_end0:
	.size	"julia_+_3126", .Lfunc_end0-"julia_+_3126"
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


In [15]:
@code_native binary = true 1.0 + 1.0

	.text
	.file	"+"
	.globl	"julia_+_3161"                  # -- Begin function julia_+_3161
	.p2align	4, 0x90
	.type	"julia_+_3161",@function
"julia_+_3161":                         # @"julia_+_3161"
; ┌ @ float.jl:383 within `+`
	.cfi_startproc
# %bb.0:                                # %top
	vaddsd	%xmm1, %xmm0, %xmm0             # encoding: [0xc5,0xfb,0x58,0xc1]
	retq                                    # encoding: [0xc3]
.Lfunc_end0:
	.size	"julia_+_3161", .Lfunc_end0-"julia_+_3161"
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


- Julia's type system is **dynamic**.


- In a statically typed language, such as C, C++, and Fortran, every variable has a type which must be specified in advance by the programmer. For example, in C, `int a;`.


- In a dynamically typed language, such as Python and Julia, values have a type only at runtime, and do not have to be specified in advance. This can save time for the programmer, and make your code more generic (it can work with many different kinds of runtime types).


- **Since Julia is dynamically typed, types are associated with values, not variables.** A variable is simply a name "bound" to a value.

### 1.1. Abstract and Concrete Types

- We said earlier that all Julia types belong to "a single, fully connected type graph, all nodes of which are equally first-class as types."


- We can further classify the types in the type graph as either:
    1. **Abstract**, or
    2. **Concrete**.


- **Key Fact 1**: All Julia objects are instances of a concrete type.

In [16]:
typeof(1.0)

Float64

In [17]:
isconcretetype(typeof(1.0))

true

In [18]:
is_instance_of_concrete_type(x) = isconcretetype(typeof(x))  # Returns true for all x

is_instance_of_concrete_type (generic function with 1 method)

In [19]:
is_instance_of_concrete_type(Float64)

true

- In turn, all concrete types are subtypes of at least one abstract type.

In [20]:
supertype(typeof(1.0))

AbstractFloat

In [21]:
isabstracttype(AbstractFloat)

true

In [ ]:
print_tree(Any)

- The abstract type `Any` sits at the top of the type tree. By definition, all types, both abstract and concrete, are subtypes of `Any`.


- We can use the operator `<:`, meaning "is a subtype of" to test whether one type is a subtype of another type.

In [23]:
is_subtype_of_Any(x) = typeof(x) <: Any  # Returns true for all x

is_subtype_of_Any (generic function with 1 method)

In [24]:
is_subtype_of_Any(Float64)

true

- **Key Fact 2**: Concrete types may not subtype each other. 


- In other words, **concrete types are final**, and only abstract types may be supertypes.

In [25]:
has_abstract_supertype(x) = isabstracttype(supertype(typeof(x)))  # Returns true for all x

has_abstract_supertype (generic function with 1 method)

- **Key Fact 3**: Abstract types cannot be instantiated. 


- (This is really the same as Key Fact 1, but it's worth emphasising again.)


- Therefore, abstract types serve only to describe relationships between related concrete types within Julia's type hierarchy. 


- Abstract types are branches in the type tree while concrete types are leaves.

In [26]:
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


- Why do we even need abstract types then, if they can never be instantiated?


- A well-defined type hierarchy, used in combination with **methods** and **multiple dispatch**, allows you to define behaviour for entire families of related concrete types. More on this later.

### 1.2. Defining Abstract Types

- You can define your own abstract types using the `abstract type` keyword.

In [27]:
abstract type Animal end

- In this context, we can use the `<:` operator - "is a subtype of" - to indicate relationships between types and thereby construct a user-defined type hierarchy.

In [28]:
abstract type Feline <: Animal end
abstract type Canine <: Animal end

In [29]:
print_tree(Animal)

Animal
├─ Canine
└─ Feline


- Remember, we can't instantiate abstract types, so we still can't create any animals. 


## 2. Composite Types (structs)  <a class="anchor" id="composite-types"></a>

- How can we define our own concrete types? 


- The most common user-defined type in Julia is a **composite type** or `struct`. This is the Julia equivalent of a `Class` in Python.


### 2.1. Defining Composite Types

- Composite types are introduced with the `struct` keyword followed by a block of field names.

In [30]:
struct Foo
    baz
    qux
    quux
end

In [31]:
foo = Foo("a", "b", "c")

Foo("a", "b", "c")

- You can also add type annotations to struct definitions, using the `::` operator (meaning "is a").

In [32]:
struct Bar
    baz  # implicit ::Any
    qux::Float64
    quux::Int64
end

In [33]:
bar = Bar("a", "b", "c")

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at number.jl:6
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at number.jl:7
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:273
[0m  ...

- Doing this can make your code readable, ensure correctness, and has implications for performance (more on this later). However, it can also make your code less flexible if you include overly restrictive type annotations.


- Whenever you omit an explicit type annotation on the field of a struct, `::Any` is implied.


- As with the definition of abstract types, we can use the `<:` operator - "is a subtype of" - to indicate that a composite type is a subtype of a specific abstract type.

In [34]:
struct Dog <: Canine
    name
    home
end

struct Wolf <: Canine
    name
    pack
end

struct Cat <: Feline
    name
    home
end

struct Lion <: Feline
    name
    pride
end

In [35]:
print_tree(Animal)

Animal
├─ Canine
│  ├─ Dog
│  └─ Wolf
└─ Feline
   ├─ Cat
   └─ Lion


### 2.2. Instantiating Composite Types

- To create a new object of a given type, simply apply the type name like a function.

In [36]:
sammy = Dog("Sammy", "Schmidt Family")

Dog("Sammy", "Schmidt Family")

In [37]:
fieldnames(Dog)

(:name, :home)

- When we call the type name like this, we are actually calling a **constructor** for that type, which is a function which returns an instance of the given type.


- Two default constructors are created automatically every time we declare a new composite type:
    1. One which accepts any arguments and attempts to convert them to the types of the fields.
    2. Another which accepts arguments matching the field types exactly.

In [38]:
struct A
    a::Float32
    b::Float64
end

In [39]:
A(1.0, 1.0)

A(1.0f0, 1.0)

In [41]:
typeof(1.0f0)

Float32

In [42]:
typeof(1.0e0)

Float64

### 2.3. Accessing the Fields of a Composite Type

- You can access the field names of a composite type using the standard `foo.bar` notation.

In [43]:
sammy.name

"Sammy"

In [44]:
sammy.home

"Schmidt Family"

### 2.4. Structs are Not Mutable

- Composite objects declared with `struct` are **immutable**; they cannot be modified after construction.

In [45]:
fieldnames(Dog)

(:name, :home)

In [47]:
sammy.name

"Sammy"

In [48]:
sammy.name = "Good Boy"

LoadError: setfield!: immutable struct of type Dog cannot be changed

- **N.B.** An immutable object might contain mutable objects, such as arrays, as fields. Those mutable field values will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

In [49]:
struct ArrayStruct
    a::Array
end
array_struct = ArrayStruct([1.0, 2.0])
array_struct.a

2-element Vector{Float64}:
 1.0
 2.0

In [50]:
# Modify the mutable object
array_struct.a[1] = 3.0
array_struct.a

2-element Vector{Float64}:
 3.0
 2.0

In [51]:
# Attempt to modify the field value itself
array_struct.a = [3.0, 4.0]

LoadError: setfield!: immutable struct of type ArrayStruct cannot be changed

- Why is it like this? Isn't this unnecessarily restrictive for the programmer?


- The main reason for structs to be immutable is to allow the compiler to effectively optimise your code. In particular, it allows your objects to be stored efficiently in memory. (Hopefully) more on this in a later lecture.


- If you really do need a mutable struct, you can simply declare one with the `mutable` keyword:

In [52]:
mutable struct MyMutableStruct
    a
end
m = MyMutableStruct("original value")
m.a

"original value"

In [53]:
m.a = "new value"
m.a

"new value"

- Be warned that using mutable structs will almost certainly impact the performance of your program and should be avoided in performane critical code.


- If you find yourself regularly reaching for mutable structs, it may be because you are still transitioning to the "Julian" way of thinking.

### 2.5. Avoid Abstract Type Annotations

- To enable Julia to compile your source code into highly efficient machine code, it is strongly advised to avoid abstract type annotations in performance critical code. 


- For example, don't do this:

In [54]:
struct TwoThings
    a
    b
end

function multiply(two_things)
    return two_things.a * two_things.b
end

two_things = TwoThings(3.0, 4.0)

@btime multiply(two_things)

  26.610 ns (1 allocation: 16 bytes)


12.0

- Or this:

In [55]:
struct TwoAbstractFloats
    a::AbstractFloat
    b::AbstractFloat
end

two_floats = TwoAbstractFloats(3.0, 4.0)

@btime multiply(two_floats)

  27.751 ns (1 allocation: 16 bytes)


12.0

- What if we had used concrete type annotations instead?

In [56]:
struct TwoFloat64s
    a::Float64
    b::Float64
end

two_float64s = TwoFloat64s(3.0, 4.0)

@btime multiply(two_float64s)

  16.382 ns (1 allocation: 16 bytes)


12.0

In [57]:
struct TwoFloat32
    a::Float32
    b::Float32
end

- Even for such a trivial operation, we can see that there is a significant impact on speed due to the abstract type annotations.


- However, we also don't want to create a new, distinct composite type for every possible combination of concrete field types. Fortunately, Julia's parametric type system provides convenient syntax for defining entire families of structs with concrete field types.

## 3. Parametric Types <a class="anchor" id="parametric-types"></a>

- We just saw how to define composite types, or structs. For example, to define a point in 2D Cartesian space we could do:

In [58]:
struct PointGeneric
    x
    y
end

- In addition, we saw that we can annotate the struct's fields with types, either abstract or concrete, and that this can have consequences for the performance of our code. For example:

In [59]:
struct PointFloat64
    x::Float64
    y::Float64
end

In [60]:
function distance(point)
    (; x, y) = point
    return sqrt(x^2 + y^2)
end

distance (generic function with 1 method)

In [61]:
p1 = PointGeneric(3.0, 4.0)
@btime distance(p1)

  77.641 ns (4 allocations: 64 bytes)


5.0

In [62]:
p2 = PointFloat64(3.0, 4.0)
@btime distance(p2)

  16.170 ns (1 allocation: 16 bytes)


5.0

- Creating a new `Point`-like type for every possible concrete type its fields could take is of course tedious and restrictive. Do not write code like this!

### 3.1. Parametric Composite Types 

- Julia's type system is **parametric**: types can take parameters, such that type declarations actually introduce an entire family of new types - one for each possible combination of the parameter values.


- We can think of this like a mathematical function with parameters, e.g. $f(x, y \,; a, b)$.


- Type parameters are introduced immediately after the type name, surrounded by curly braces:

In [63]:
struct Point{T}
   x::T
   y::T
end

- `T`, the type parameter, can be *any* Julia type.


- This particular declaration defines a new parametric type, `Point{T}`, holding two "coordinates", each of the same type `T`.


- For example, `Point{Float64}` is a concrete type equivalent to the type defined by replacing `T` in the definition of `Point` with `Float64`. That is, `Point{Float64}` is equivalent to the `PointFloat64` struct that we defined earlier.

In [65]:
isconcretetype(Point{Float64})

true

In [67]:
Point{String}

Point{String}

In [68]:
p3 = Point{Any}(3.0, 4.0)
@btime distance(p3)

  79.734 ns (4 allocations: 64 bytes)


5.0

In [69]:
p4 = Point{Float64}(3.0, 4.0)
@btime distance(p4)

  16.015 ns (1 allocation: 16 bytes)


5.0

In [70]:
isconcretetype(Point{Float64})

true

In [72]:
isabstracttype(Point)

false

In [73]:
isstructtype(Point)

true

In [74]:
p3

Point{Any}(3.0, 4.0)

In [75]:
p4

Point{Float64}(3.0, 4.0)

In [76]:
typeof(p3)

Point{Any}

In [77]:
typeof(p4)

Point{Float64}

- We see therefore that our parametric definition of `Point{T}` actually declares an unlimited number of types, one for each possible value of `T`, each of which is now a usable concrete type. 

- When creating an instance of `Point`, the value of the parameter type `T` can be omitted if it is unambiguous:

In [80]:
Point(1.0, 2.0)

Point{Float64}(1.0, 2.0)

In [81]:
Point("1.0", "2.0")

Point{String}("1.0", "2.0")

- `Point` itself is a valid type object (neither abstract nor concrete!) containing all possible instances of `Point{T}` as subtypes:

In [83]:
Point{Float64} <: Point

true

In [85]:
Point{Any} <: Point

true

- However, concrete `Point` types with different values of `T` are never subtypes of each other:

In [88]:
Point{Float64} <: Point{AbstractFloat}

false

In [87]:
Float64 <: AbstractFloat

true

- This is because Julia's type system is **invariant**, which means that, for any three types `A`, `B`, `C`, where `C` is parametrised by another type, `A <: B` does not imply that `C{A} <: C{B}`.

- Often, we will want to restrict the possible values of the type parameter `T`. For example, to restrict our coordinates to real numbers, we could have done:

In [89]:
struct PointReal{T<:Real}
   x::T
   y::T
end

In [90]:
PointReal{String}

LoadError: TypeError: in PointReal, in T, expected T<:Real, got Type{String}

In [91]:
function count_subtypes(T)
    count = 0
    
    for subtype in subtypes(T)
        count += 1
        if !isconcretetype(subtype)
            count += count_subtypes(subtype)
        end
    end
    
    return count
end

count_subtypes (generic function with 1 method)

In [92]:
count_subtypes(Real)

23

In [93]:
print_tree(Real)

Real
├─ AbstractFloat
│  ├─ BigFloat
│  ├─ Float16
│  ├─ Float32
│  └─ Float64
├─ AbstractIrrational
│  └─ Irrational
├─ Integer
│  ├─ Bool
│  ├─ Signed
│  │  ├─ BigInt
│  │  ├─ Int128
│  │  ├─ Int16
│  │  ├─ Int32
│  │  ├─ Int64
│  │  └─ Int8
│  └─ Unsigned
│     ├─ UInt128
│     ├─ UInt16
│     ├─ UInt32
│     ├─ UInt64
│     └─ UInt8
└─ Rational


- We can also have multiple type parameters for a single parametric type. For example:

In [94]:
struct Person{A<:AbstractString, B<:Integer, C<:Real}
    name::A
    age::B
    height::C
    weight::C
end

In [96]:
Person("Alistair", 30, 1.0, "1.0")

LoadError: MethodError: no method matching Person(::String, ::Int64, ::Float64, ::String)
[0mClosest candidates are:
[0m  Person(::A, ::B, ::C, [91m::C[39m) where {A<:AbstractString, B<:Integer, C<:Real} at In[94]:2

### 3.2. Parametric Abstract Types

- Parametric abstract type declarations declare a collection of abstract types, in much the same way:

In [97]:
abstract type Pointy{T} end

- Once again, our parametric type declaration defines an unlimited number of abstract types, one for every possible value of `T`.


- We could have declared `Point{T}` to be a subtype of `Pointy{T}`. For example:

In [98]:
struct Point2D{T} <: Pointy{T}
   x::T
   y::T
end

In [99]:
struct Point3D{T} <: Pointy{T}
   x::T
   y::T
   z::T
end

- We now have a distinct type tree for each allowed (real) value of `T`. For example:

In [100]:
print_tree(Pointy{Float64})

Pointy{Float64}
├─ Point2D{Float64}
└─ Point3D{Float64}


In [101]:
print_tree(Pointy{Int32})

Pointy{Int32}
├─ Point2D{Int32}
└─ Point3D{Int32}


In [102]:
print_tree(Pointy)

Pointy
├─ Point2D
└─ Point3D


## Summary So Far

1. Types in Julia can be abstract or concrete.
2. A composite type is introduced with the `struct` keyword and consists of a block of field names along with optional type annotations.
3. Structs are immutable.
4. Abstract type annotations can slow down your code.
5. Parametric types define entire families of types.

## 4. Methods and Multiple Dispatch <a class="anchor" id="methods"></a>

### 4.1 Methods


- Remember from the first lecture that a function maps a tuple of arguments to a return value.


- From the Julia documentation (emphasis mine):


> It is common for the same conceptual function or operation to be implemented quite differently for **different types of arguments**: adding two integers is very different from adding two floating-point numbers, both of which are distinct from adding an integer to a floating-point number. Despite their implementation differences, these operations all fall under the general concept of "addition". Accordingly, in Julia, these behaviors all belong to a single object: the + function.


- Conceptually, we have one function (addition), with many possible behaviours, depending on the types of the inputs (integers, floats, matrices, and so on).


- **A definition of one possible behaviour for a function, given the number and types of its arguments, is called a method.**

In [103]:
methods(+)

# 206 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383
[3] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:531
[4] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:537
[5] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:414
[8] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/periods.jl:378
[9] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Hermitian) in LinearAlgebra at /opt/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:469
[10] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.SymTridiagonal{var"#s882", V} where {var"#s882"<:Real, V<:AbstractVector{var"#s882"}}) in LinearAlgebra at /opt/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:488
[11] +(H::LinearAlgebra.Hermitian, D::LinearAlgebra.Diagonal{var"#s885", V} where {var"#s885"<:Real, V<:AbstractVector{var"#s885"}}) in LinearAlgebra at /opt/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:197
[12] +(A::LinearAlgebra.Hermitian, J::LinearAlgebra.UniformScaling{<:Complex}) in LinearAlgebra at /opt/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:195
[13] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /opt/julia-1.8.2/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:15
[14] +(A::LinearAlgebra.Hermitian, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /opt/julia-1.8.2/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:18
[15] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Symmetric{<:Real, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /opt/julia-1.8.2/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:26
[16] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Symmetric{var"#s885", S} where {var"#s885"<:Real, S<:(AbstractMatrix{<:var"#s885"})}) in LinearAlgebra at /opt/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:483
[17] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /opt/julia-1.8.2/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:24
[18] +(x::P, y::P) where P<:Dates.Period in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/periods.jl:77
[19] +(x::Dates.Period, y::Dates.Period) in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/periods.jl:367
[20] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/periods.jl:369
[21] +(y::Dates.Period, x::Dates.TimeType) in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/arithmetic.jl:85
[22] +(x::Dates.Period, r::AbstractRange{<:Dates.TimeType}) in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/ranges.jl:63
[23] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{<:Dates.TimeType}) in Dates at /opt/julia-1.8.2/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:14
[24] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /opt/julia-1.8.2/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:14
[25] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Hermitian) in SparseArrays at /opt/julia-1.8.2/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:17
[26] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /opt/julia-

In [104]:
function f(a, b = 1)
    a + b
end

f (generic function with 2 methods)

In [105]:
methods(f)

# 2 methods for generic function "f":
[1] f(a) in Main at In[104]:1
[2] f(a, b) in Main at In[104]:1

### 4.2. Multiple Dispatch

- Given a single function with many methods, how does Julia choose which method to call?

> The choice of which method to execute when a function is applied is called dispatch. **Julia allows the dispatch process to choose which of a function's methods to call based on the number of arguments given, and on the types of all of the function's arguments**. This is different than traditional object-oriented languages, where dispatch occurs based only on the first argument, which often has a special argument syntax, and is sometimes implied rather than explicitly written as an argument. **Using all of a function's arguments to choose which method should be invoked, rather than just the first, is known as multiple dispatch**. 


- In particular, Julia will automatically select the **most specific** method matching the arguments provided.


- What do we mean by most specific? Multiple dispatch can work on both abstract types and concrete types, and subtypes are considered more specific than their supertypes. 


- This is why a well-defined type hierarchy and multiple dispatch go hand-in-hand.


### 4.3. Defining Methods


- To define new methods, simply use the same function definition syntax we saw in the first lecture:
    1. The first time you define a function with a given name, the function object will be created along with a single method.
    2. Each subsequent function declaration with the same function name and distinct arguments (distinct by either number or type) will create a new method associated with the existing function.

In [106]:
function add(x, y)
    x + y
end

add (generic function with 1 method)

In [107]:
methods(add)

# 1 method for generic function "add":
[1] add(x, y) in Main at In[106]:1

In [108]:
function add(x::Float64, y::Float64)
    x + y
end

add (generic function with 2 methods)

In [109]:
methods(add)

# 2 methods for generic function "add":
[1] add(x::Float64, y::Float64) in Main at In[108]:1
[2] add(x, y) in Main at In[106]:1

In [110]:
function add(x::Float64, y::Float32)
    x + y
end

add (generic function with 3 methods)

In [111]:
methods(add)

# 3 methods for generic function "add":
[1] add(x::Float64, y::Float64) in Main at In[108]:1
[2] add(x::Float64, y::Float32) in Main at In[110]:1
[3] add(x, y) in Main at In[106]:1

In [112]:
function add(x, y, z)
    x + y + z
end

add (generic function with 4 methods)

In [113]:
methods(add)

# 4 methods for generic function "add":
[1] add(x::Float64, y::Float64) in Main at In[108]:1
[2] add(x::Float64, y::Float32) in Main at In[110]:1
[3] add(x, y) in Main at In[106]:1
[4] add(x, y, z) in Main at In[112]:1

In [117]:
add(1, 2, 3)

6

In [116]:
add(1, 2f0)

3.0f0

- Note that, **unlike type annotations on structs, type annotations do not have impact on the performance of methods**. 


- Instead, each time you call a function with a new combination of arguments for the first time, Julia will automatically compile an efficient version of that function for those specific argument types. Each subsequent invocation of the function with the same argument types will use the already compiled method.

In [119]:
methods(add)

# 4 methods for generic function "add":
[1] add(x::Float64, y::Float64) in Main at In[108]:1
[2] add(x::Float64, y::Float32) in Main at In[110]:1
[3] add(x, y) in Main at In[106]:1
[4] add(x, y, z) in Main at In[112]:1

In [120]:
@btime add(1f0, 2f0)  # 32-bit floats

  1.305 ns (0 allocations: 0 bytes)


3.0f0

In [121]:
function add(x::Float32, y::Float32)
    x + y
end

add (generic function with 5 methods)

In [122]:
methods(add)

# 5 methods for generic function "add":
[1] add(x::Float64, y::Float64) in Main at In[108]:1
[2] add(x::Float64, y::Float32) in Main at In[110]:1
[3] add(x::Float32, y::Float32) in Main at In[121]:1
[4] add(x, y) in Main at In[106]:1
[5] add(x, y, z) in Main at In[112]:1

In [123]:
@btime add(1f0, 2f0)

  1.303 ns (0 allocations: 0 bytes)


3.0f0

### 4.4. Example: Animals

- Earlier, we defined some animals. Now, let's add behaviour using methods and multiple dispatch.

In [124]:
print_tree(Animal)

Animal
├─ Canine
│  ├─ Dog
│  └─ Wolf
└─ Feline
   ├─ Cat
   └─ Lion


In [125]:
fieldnames(Dog)

(:name, :home)

In [126]:
fieldnames(Wolf)

(:name, :pack)

In [127]:
fieldnames(Cat)

(:name, :home)

In [128]:
fieldnames(Lion)

(:name, :pride)

In [129]:
# Create some animals
teddy = Dog("Teddy", "Fischer Family")
yukon = Wolf("Yukon", "Inner Alaska Pack")
felix = Cat("Felix", "Müller Family")
simba = Lion("Simba", "Mount Elgon Pride");

In [130]:
# Methods can be annotated with abstract types
function encounter(canine_one::Canine, canine_two::Canine)
    println("Uncertain whether these two canines will get along. Remain wary.")
end

# Methods without type annotations will accept any type
function encounter(animal_one, animal_two)
    println("No well defined encounter for a $(typeof(animal_one)) and a $(typeof(animal_two)).")
end

encounter (generic function with 2 methods)

In [131]:
methods(encounter)

# 2 methods for generic function "encounter":
[1] encounter(canine_one::Canine, canine_two::Canine) in Main at In[130]:2
[2] encounter(animal_one, animal_two) in Main at In[130]:7

In [132]:
encounter(teddy, yukon)

Uncertain whether these two canines will get along. Remain wary.


In [133]:
encounter(teddy, felix)

No well defined encounter for a Dog and a Cat.


In [134]:
# Define more specific methods annotated with concrete types
function encounter(dog_one::Dog, dog_two::Dog)
    println("$(dog_one.name) wags tail. So does $(dog_two.name).")
end

function encounter(cat_one::Cat, cat_two::Cat)
    println("No thanks.")
end

function encounter(dog::Dog, cat::Cat)
    println("$(cat.name) chases $(dog.name) away.")
end

encounter(cat::Cat, dog::Dog) = play(dog, cat)  # Allow arguments to be provided in either order

encounter (generic function with 6 methods)

In [135]:
methods(encounter)

# 6 methods for generic function "encounter":
[1] encounter(dog_one::Dog, dog_two::Dog) in Main at In[134]:2
[2] encounter(canine_one::Canine, canine_two::Canine) in Main at In[130]:2
[3] encounter(cat_one::Cat, cat_two::Cat) in Main at In[134]:6
[4] encounter(dog::Dog, cat::Cat) in Main at In[134]:10
[5] encounter(cat::Cat, dog::Dog) in Main at In[134]:14
[6] encounter(animal_one, animal_two) in Main at In[130]:7

In [136]:
encounter(teddy, felix)

Felix chases Teddy away.


In [137]:
encounter(simba, felix)

No well defined encounter for a Lion and a Cat.


In [138]:
encounter(felix, felix)

No thanks.


- **Multiple dispatch is one of the core programming paradigms of the Julia language.**


- How does this compare to standard object-oriented languages? How would you do same example with the animals and their encounters in Python?



- **Tip**: To get the most out of Julia's multiple dispatch, avoid the temptation to define overly specific functions. For example, you could define a method `encounter` for *any* objects which can do a thing called encounter. Add methods to the function `encounter` instead of defining new functions like `encounter_dog_dog` or `encounter_dog_dog_dog`.

## 5. Parametric Methods <a class="anchor" id="parametric-methods"></a>


- Type parameters can also be used in method definitions. For example:

In [139]:
function f(x::T, y::T) where {T}
    return true
end

function f(x, y)
    return false
end

f (generic function with 3 methods)

In [140]:
f(1.0, 1.0)

true

In [141]:
f(1.0, 1.0f0)

false

- What do you think this function does?


- As before, the values of `T` can be restricted to subtypes of a given type. For example:

In [143]:
function f_numeric(x::T, y::T) where {T<:Number}
    return true
end

function f_numeric(x, y)
    return false
end

f_numeric (generic function with 2 methods)

- What do you think this function does?

In [145]:
f_numeric("1.0", "1.0")

false

- The method type parameter `T` can also be used inside the function body:

In [146]:
function g(x::T) where T
    return T
end

g (generic function with 1 method)

- What do you think this function does?

In [148]:
g(1.0)  # typeof(1.0)

Float64

In [149]:
g("1")

String

- As before, multiple type parameters are possible:

In [150]:
function concat_number_to_string(x::S, y::N) where {S<:AbstractString, N<:Number}
    return x * string(y)
end

function concat_number_to_string(x::N, y::S) where {N<:Number, S<:AbstractString}
    return string(x) * y
end

concat_number_to_string (generic function with 2 methods)

In [151]:
concat_number_to_string("My number is ", 1.0)

"My number is 1.0"

In [152]:
concat_number_to_string(1.0, " is my number.")

"1.0 is my number."

- Why are parametric methods useful? One use-case is to ensure the **correctness** of your code, by enforcing invariants.

In [153]:
function strict_add(x::T, y::T) where {T}
    x + y
end

strict_add (generic function with 1 method)

In [154]:
strict_add(1f0, 2e0)

LoadError: MethodError: no method matching strict_add(::Float32, ::Float64)
[0mClosest candidates are:
[0m  strict_add(::T, [91m::T[39m) where T at In[153]:1

In [155]:
@code_lowered 1f0 + 1f0

CodeInfo(
1 ─ %1 = Base.add_float(x, y)
└──      return %1
)

In [156]:
@code_lowered 1.0f0 + 2.0e0

CodeInfo(
1 ─ %1 = Base.promote(x, y)
│   %2 = Core._apply_iterate(Base.iterate, Base.:+, %1)
└──      return %2
)

## 6. Conclusion <a class="anchor" id="multiple-dispatch"></a>

- When you need to define custom structure and behaviour in Julia - which will be often - think about:  
    1. Structs (for structure).
    2. Methods which operate on your structs (for behaviour).

## 7. Further Reading <a class="anchor" id="further-reading"></a>

- The [official Julia documentation](https://docs.julialang.org/en/v1/) is quite thorough and well-explained on the topics we've covered today:  

    1. [Types](https://docs.julialang.org/en/v1/manual/types/)
    2. [Methods and Multiple Dispatch](https://docs.julialang.org/en/v1/manual/methods/)
    3. [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)
    
    

- [JuliaCon 2019 | The Unreasonable Effectiveness of Multiple Dispatch | Stefan Karpinski](https://www.youtube.com/watch?v=kc9HwsxE1OY)


## 8. Next Week <a class="anchor" id="next-week"></a>
1. Chaos in Dynamical Systems, Part I
2. [DynamicalSystems.jl](https://juliadynamics.github.io/DynamicalSystems.jl/latest/), Part I

## 9. Quick Survey  <a class="anchor" id="survey"></a>

- Please take three minutes to fill out this survey on the course so far: https://forms.gle/SdDGr1E46v89dSBX8


- The more feedback we get, the more we can tailor the course to you!